In [1]:
import requests
from bs4 import BeautifulSoup
import pandas as pd

Получаем данные таблицы

In [2]:
url = 'https://nationalbank.kz/ru/exchangerates/ezhednevnye-oficialnye-rynochnye-kursy-valyut'
response = requests.get(url)
if response.status_code == 200:
    soup = BeautifulSoup(response.text, 'html.parser')

Находим нужные ячейки таблицы

In [4]:
table = soup.find('table')
rows = table.find_all('tr')

currencies_rates = []
currencies_code = []
currencies_name = []
currencies_quant = []
        
for row in rows:
    input_tag = row.find('input', {'id': 'rates'})
    if input_tag is not None:
        rate_value = input_tag['value']
    currencies_rates.append(rate_value)

    table_data = row.find_all('td')
    for td in table_data:
        if 'text-left' in td.get('class', []):
            currency_info = td.text.strip()
            quantity = int(currency_info.split()[0])
            currency_name = ' '.join(currency_info.split()[1:])
            currencies_quant.append(quantity)
            currencies_name.append(currency_name)
        if 'KZT' in td.text:
            currency_code = td.text.split('/')[0].strip()
            currencies_code.append(currency_code)


In [6]:
if len(currencies_code) == len(currencies_rates) == len(currencies_quant) == len(currencies_name):
    data = {'Quantity': currencies_quant, 'CurrencyName': currencies_name, 'CurrencyCode': currencies_code, 'Rate': currencies_rates}
df = pd.DataFrame(data)

In [7]:
df.head()

,Quantity,CurrencyName,CurrencyCode,Rate
0,1,АВСТРАЛИЙСКИЙ ДОЛЛАР,AUD,1
1,1,АЗЕРБАЙДЖАНСКИЙ МАНАТ,AZN,48
2,10,АРМЯНСКИЙ ДРАМ,AMD,51
3,1,БЕЛОРУССКИЙ РУБЛЬ,BYN,38
4,1,БРАЗИЛЬСКИЙ РЕАЛ,BRL,46
